<a href="https://colab.research.google.com/github/JollyFoxy/OBD/blob/main/Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Нейронная сеть с несколькими слоями

## Подключаем библиотеки

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Загружаем данные

In [5]:
dataset = pd.read_csv('data.csv')
X = dataset.iloc[:,3:13].values
y = dataset.iloc[:,13].values

In [6]:
X

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [7]:
y

array([1, 0, 1, ..., 1, 1, 0])

## Кодируем категорийные данные

In [8]:
!pip install OneHotEncoder

ERROR: Could not find a version that satisfies the requirement OneHotEncoder (from versions: none)
ERROR: No matching distribution found for OneHotEncoder


In [9]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [10]:
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])

labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])

#onehotencoder = OneHotEncoder(categorical_features = [1]) Not this one

# use this instead
ct = ColumnTransformer([("Country", OneHotEncoder(), [1])], remainder = 'passthrough')

X = ct.fit_transform(X)
X = X[:, 1:]

In [11]:
X.shape

(10000, 11)

In [12]:
y

array([1, 0, 1, ..., 1, 1, 0])

## Разбиваем на обучающую и тестовую выборки

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state = 0)

In [14]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Импортируем модуль keras

In [15]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

## Инициализация нейронной сети 

In [16]:
classifier = Sequential()

## Добавляем входной слой и первый скрытый слой

In [17]:
classifier.add(Dense(units = 6, activation ='relu', input_dim = 11, kernel_initializer = 'uniform'))

## Второй скрытый слой

In [18]:
classifier.add(Dense(units = 6, activation = 'relu', kernel_initializer = 'uniform'))

## Выходной слой

In [19]:
classifier.add(Dense(units = 1, activation = 'sigmoid', kernel_initializer = 'uniform'))

## Компилируем нейронную сеть

In [20]:
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

## Применяем к тренировочной выборке

In [21]:
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4915 - accuracy: 0.7951
Epoch 2/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4272 - accuracy: 0.7960
Epoch 3/100
800/800 [==============================] - 1s 976us/step - loss: 0.4201 - accuracy: 0.8145
Epoch 4/100
800/800 [==============================] - 1s 985us/step - loss: 0.4135 - accuracy: 0.8319
Epoch 5/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4091 - accuracy: 0.8339
Epoch 6/100
800/800 [==============================] - 2s 2ms/step - loss: 0.4058 - accuracy: 0.8345
Epoch 7/100
800/800 [==============================] - 2s 2ms/step - loss: 0.4044 - accuracy: 0.8332
Epoch 8/100
800/800 [==============================] - 2s 2ms/step - loss: 0.4028 - accuracy: 0.8340
Epoch 9/100
800/800 [==============================] - 2s 2ms/step - loss: 0.4022 - accuracy: 0.8340
Epoch 10/100
800/800 [==============================] - 2s 2ms/step - loss: 0.4016 - ac

## Прогноз

In [22]:
y_pred = classifier.predict(X_test)
y_pred

array([[0.29684234],
       [0.3148399 ],
       [0.16455871],
       ...,
       [0.20737931],
       [0.21227959],
       [0.18492135]], dtype=float32)

In [23]:
y_pred = (y_pred > 0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

##  Матрица несоответствий

In [24]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[1507,   88],
       [ 188,  217]])

## Прогноз для клиента с данными

In [25]:
new_prediction = classifier.predict(sc.transform(np.array([[0.0,0,600,1,40,3,60000,2,1,1,50000]])))

In [26]:
new_prediction

array([[0.03002116]], dtype=float32)